# Leeds property price analytics

In [36]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# force full-width display if viewed in Chrome browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## Download UK Land Registry price-paid data and create bigquery dataset

In [ ]:
# download data to the VM from source: https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads
!wget http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv

In [ ]:
# move to gcp bucket
!gsutil mv pp-complete.csv gs://housingasodhfmq349p78vp57pasvpfphio/pp-complete.csv

In [ ]:
# make bigquery database
!bq mk housing

In [ ]:
# create table called 'pricepaid' in housing database
!bq load --source_format=CSV \
    housing.pricepaid gs://housingasodhfmq349p78vp57pasvpfphio/pp-complete.csv \
    trans_id:STRING,price:INTEGER,date:STRING,postcode:STRING,type:STRING,newbuild:STRING,tenure:STRING,paon:STRING,saon:STRING,street:STRING,locality:STRING,town_city:STRING,district:STRING,county:STRING,ppd_type:STRING,record_status:STRING

## Query data (Leeds only)

In [265]:
%%bigquery df

SELECT * FROM housing.pricepaid
WHERE town_city = 'LEEDS'

Downloading: 100%|██████████| 302956/302956 [00:02<00:00, 119689.57rows/s]


## Inspect dataframe

In [266]:
df.head(4)

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status
0,{80B98B64-F7A8-4F75-8D6B-CF92348EA779},187000,2004-01-30 00:00,LS17 7SH,D,N,F,17,,SUNNINGDALE DRIVE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A
1,{21E5FEB5-D94C-2439-E050-A8C06205342E},267000,2014-08-26 00:00,LS7 4LY,D,N,F,1C,,CHURCH LANE,CHAPEL ALLERTON,LEEDS,LEEDS,WEST YORKSHIRE,B,A
2,{DFCBEA35-78ED-4CB1-B892-119778FCEEA5},304000,2015-09-17 00:00,LS17 8XR,D,N,F,19,,OAKDENE WAY,,LEEDS,LEEDS,WEST YORKSHIRE,A,A
3,{5BD5D826-4B53-4D3E-852A-8DA733041E3C},66000,1998-10-02 00:00,LS12 5LX,D,N,F,2,,WATER LANE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A


In [267]:
df.nunique()

trans_id         302956
price             12063
date               7702
postcode          14067
type                  5
newbuild              2
tenure                3
paon               6205
saon               2348
street             8956
locality            120
town_city             1
district             11
county                4
ppd_type              2
record_status         1
dtype: int64

In [268]:
df['ppd_type'].unique()

array(['A', 'B'], dtype=object)

In [269]:
# check data types
df.dtypes

trans_id         object
price             int64
date             object
postcode         object
type             object
newbuild         object
tenure           object
paon             object
saon             object
street           object
locality         object
town_city        object
district         object
county           object
ppd_type         object
record_status    object
dtype: object

## Pre-processing

#### 1. Postcode

In [270]:
# Some rows have missing postcodes
len(df[df['postcode'] == ''])

202

In [271]:
# Check for null entries in postcode column
len(df[df['postcode'].isnull()])

0

In [272]:
# Discard rows with empty string postcodes 
df = df[df['postcode'] != '']

In [273]:
# Check that all postcodes have space
len(df[df['postcode'].str.contains(' ')]) == len(df)

True

In [274]:
# Break down postcodes into components: area, district, sector ,and unit.
def get_postcode_component(postcode, component):
    
    """
    Returns part of all of a postcode string resolved into area, district, sector or unit. 
    
    UK postcodes take any of the following formats, where A denotes alphabetical
    and N denotes numeric:
    
    AN NAA 
    ANN NAA
    AAN NAA
    AANN NAA
    ANA NAA
    AANA NAA
    
    (source: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/611951/Appendix_C_ILR_2017_to_2018_v1_Published_28April17.pdf)
    
    - Area is the leftmost substring of alpha characters until (and excluding) a numeric character .Format: A or AA.
    - District is the substring commencing with numeric after the area, and before the space. Format: N, NN, or NA.
    - Sector is the first numeric character after the space. Format: N.
    - Unit is the rightmost substring. Format: AA.
   
    e.g.:
    >> get_postcode_component('RG40 2WD', component='sector')
    >> RG40 2   
    
    e.g. 2:
    >> get_postcode_component('EC1A 3HX', component='area')
    >> EC
    
    e.g. 3:
    >> get_postcode_component('EC1A 3HX', component='district')
    >> EC1A
    """
    
    # e.g. EC1A 3HX   /   RG40 3HX
    area_and_district = list(postcode.split(' ')[0])  # ['E', 'C', '1', 'A']   /   ['R', 'G', '4', '0']
    sector_and_unit = list(postcode.split(' ')[1])    # ['3', 'H', 'X']        /   ['3', 'H', 'X']
    
    if component == 'area':
        for idx, el in enumerate(area_and_district):
            if el.isnumeric():
                area = ''.join(area_and_district[:idx]) # EC  /  RG
                return area        

    elif component == 'district':
        for idx, el in enumerate(area_and_district):
            if el.isnumeric():
                district = ''.join(area_and_district[idx:]) # 1A  /  40
                return district
                
    elif component == 'sector':
        for idx, el in enumerate(sector_and_unit):
            if not el.isnumeric():
                sector = ''.join(sector_and_unit[:idx])   # 3   /   2
                return sector
                
    elif component == 'unit':
        for idx, el in enumerate(sector_and_unit):
            if not el.isnumeric():
                unit = ''.join(sector_and_unit[idx:])   # HX  /   QS
                return unit
    
    else:
        return None

In [275]:
# unit tests
test_postcodes = ['A1 2BC', 'A12 3BC', 'AB1 2BC', 'AB12 2CD', 'A1B 2BC', 'AB1C 2DE']

for postcode in test_postcodes:
    print(postcode, 'area: :', get_postcode_component(postcode, component='area'))
    print(postcode, 'district: :', get_postcode_component(postcode, component='district'))
    print(postcode, 'sector: :', get_postcode_component(postcode, component='sector'))
    print(postcode, 'unit: :', get_postcode_component(postcode, component='unit'))

A1 2BC area: : A
A1 2BC district: : 1
A1 2BC sector: : 2
A1 2BC unit: : BC
A12 3BC area: : A
A12 3BC district: : 12
A12 3BC sector: : 3
A12 3BC unit: : BC
AB1 2BC area: : AB
AB1 2BC district: : 1
AB1 2BC sector: : 2
AB1 2BC unit: : BC
AB12 2CD area: : AB
AB12 2CD district: : 12
AB12 2CD sector: : 2
AB12 2CD unit: : CD
A1B 2BC area: : A
A1B 2BC district: : 1B
A1B 2BC sector: : 2
A1B 2BC unit: : BC
AB1C 2DE area: : AB
AB1C 2DE district: : 1C
AB1C 2DE sector: : 2
AB1C 2DE unit: : DE


In [276]:
# create new columns with postcode components
df['pc_area'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'area'))
df['pc_district'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'district'))
df['pc_sector'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'sector'))
df['pc_unit'] = df['postcode'].apply(lambda x: get_postcode_component(x, 'unit'))

In [277]:
df.nunique()

trans_id         302754
price             12043
date               7699
postcode          14066
type                  5
newbuild              2
tenure                3
paon               6122
saon               2321
street             8944
locality            119
town_city             1
district             11
county                4
ppd_type              2
record_status         1
pc_area               1
pc_district          25
pc_sector            10
pc_unit             387
dtype: int64

In [286]:
df

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status,pc_area,pc_district,pc_sector,pc_unit
0,{80B98B64-F7A8-4F75-8D6B-CF92348EA779},187000,2004-01-30 00:00,LS17 7SH,D,N,F,17,,SUNNINGDALE DRIVE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,17,7,SH
1,{21E5FEB5-D94C-2439-E050-A8C06205342E},267000,2014-08-26 00:00,LS7 4LY,D,N,F,1C,,CHURCH LANE,CHAPEL ALLERTON,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,7,4,LY
2,{DFCBEA35-78ED-4CB1-B892-119778FCEEA5},304000,2015-09-17 00:00,LS17 8XR,D,N,F,19,,OAKDENE WAY,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,17,8,XR
3,{5BD5D826-4B53-4D3E-852A-8DA733041E3C},66000,1998-10-02 00:00,LS12 5LX,D,N,F,2,,WATER LANE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,12,5,LX
4,{BA77456B-DB14-4FC9-849C-481AB50ECC93},269000,2010-10-29 00:00,LS26 8WY,D,N,F,11,,ELDER RISE,WOODLESFORD,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,26,8,WY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302951,{7FD8E68D-5BB0-4963-BECF-7E0D4E284A51},193000,2007-07-26 00:00,LS25 6NQ,T,N,F,9,,SAXON GRANGE,SHERBURN IN ELMET,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,6,NQ
302952,{79A74E21-FD10-1289-E053-6B04A8C01627},188999,2018-05-31 00:00,LS25 6GG,T,N,F,70,,BRAEBURN ROAD,SHERBURN IN ELMET,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,6,GG
302953,{3E0330F0-308A-8D89-E050-A8C062052140},165500,2016-08-12 00:00,LS25 5BU,T,N,F,24,,COMMON LANE,SOUTH MILFORD,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,5,BU
302954,{2765E50C-A82F-4300-9AED-94AACCDBD16E},340000,2004-05-14 00:00,LS25 5HP,T,N,F,CROFT HOUSE,,CHAPEL STREET,HILLAM,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,5,HP


#### 2. Date

In [297]:
# convert date column to timestamp and discard meaningless hours/minutes data
df['date'] = df['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))

,trans_id,price,date,postcode,type,newbuild,tenure,paon,saon,street,locality,town_city,district,county,ppd_type,record_status,pc_area,pc_district,pc_sector,pc_unit,date_test
0,{80B98B64-F7A8-4F75-8D6B-CF92348EA779},187000,2004-01-30 00:00,LS17 7SH,D,N,F,17,,SUNNINGDALE DRIVE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,17,7,SH,2004-01-30
1,{21E5FEB5-D94C-2439-E050-A8C06205342E},267000,2014-08-26 00:00,LS7 4LY,D,N,F,1C,,CHURCH LANE,CHAPEL ALLERTON,LEEDS,LEEDS,WEST YORKSHIRE,B,A,LS,7,4,LY,2014-08-26
2,{DFCBEA35-78ED-4CB1-B892-119778FCEEA5},304000,2015-09-17 00:00,LS17 8XR,D,N,F,19,,OAKDENE WAY,,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,17,8,XR,2015-09-17
3,{5BD5D826-4B53-4D3E-852A-8DA733041E3C},66000,1998-10-02 00:00,LS12 5LX,D,N,F,2,,WATER LANE,LEEDS,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,12,5,LX,1998-10-02
4,{BA77456B-DB14-4FC9-849C-481AB50ECC93},269000,2010-10-29 00:00,LS26 8WY,D,N,F,11,,ELDER RISE,WOODLESFORD,LEEDS,LEEDS,WEST YORKSHIRE,A,A,LS,26,8,WY,2010-10-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302951,{7FD8E68D-5BB0-4963-BECF-7E0D4E284A51},193000,2007-07-26 00:00,LS25 6NQ,T,N,F,9,,SAXON GRANGE,SHERBURN IN ELMET,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,6,NQ,2007-07-26
302952,{79A74E21-FD10-1289-E053-6B04A8C01627},188999,2018-05-31 00:00,LS25 6GG,T,N,F,70,,BRAEBURN ROAD,SHERBURN IN ELMET,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,6,GG,2018-05-31
302953,{3E0330F0-308A-8D89-E050-A8C062052140},165500,2016-08-12 00:00,LS25 5BU,T,N,F,24,,COMMON LANE,SOUTH MILFORD,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,5,BU,2016-08-12
302954,{2765E50C-A82F-4300-9AED-94AACCDBD16E},340000,2004-05-14 00:00,LS25 5HP,T,N,F,CROFT HOUSE,,CHAPEL STREET,HILLAM,LEEDS,SELBY,NORTH YORKSHIRE,A,A,LS,25,5,HP,2004-05-14


In [278]:
# df = df.sort_values('date')
# 
# df['street'] = df['street'].astype(str)
# df['street'] = df['street'].apply(lambda x: x.lower())
# 
# df['days_ago'] = df['date'].apply(lambda x: datetime.today() - datetime.strptime(x, '%Y-%m-%d'))
# df['days_ago'] = df['days_ago'].apply(lambda x: str(x).split(' ')[0])
# df['days_ago'] = df['days_ago'].apply(lambda x: -int(x))
# df['months_ago'] = df['days_ago'].apply(lambda x: int(round( x/(365/12) ,0) ))

In [279]:
# def graph(postcode, housetype, streetname):
# 
#     import matplotlib.pyplot as plt
# 
#     postcode_type = df[  (df['postcode_area'] == f'{postcode}') & (df['type'] == f'{housetype}') ]
#     
#     # df for individual street
#     street = postcode_type[postcode_type['street'] == f'{streetname}']
#     
#     # price data for street
#     street_price = street.groupby(by=['months_ago']).agg({'price': 'median'})
#     
#     # moving average for street
#     street_price['MA'] = street_price['price'].rolling(window=12).mean()
#     
#     # price data for postcode area
#     postcode_type_price = postcode_type.groupby(by=['months_ago']).agg({'price': 'median'})
#     postcode_type_price = postcode_type_price.groupby(by=['months_ago']).agg({'price': 'median'})
#     
#     # moving average for postcode area
#     postcode_type_price['MA'] = postcode_type_price['price'].rolling(window=12).mean()
#     
#     plt.plot(street_price.index, street_price['price'].values,c='red',  label=f'{streetname}')
#     plt.plot(street_price.index, street_price['MA'].values, c='pink',  label=f'{streetname} moving average')
#     
#     plt.plot(postcode_type_price.index, postcode_type_price['MA'].values ,c='green',  label=f'{postcode, housetype} moving average')
#     plt.scatter(postcode_type_price.index, postcode_type_price['price'].values, c='blue', label=f'{postcode, housetype}')
#     
#     plt.rcParams["figure.figsize"]=(20,20)
#     plt.xticks(rotation=90)
#     plt.legend(loc='best')
#     plt.show()

In [280]:
def mortgage(principal, total_period, init_period, init_payment, init_rate, subs_rate, base_rate, over_payment):
    
    """
    Mortgage payment calculator. Returns dictionary of arrays
    
    principal: amount borrowed in arbitrary currency, e.g. 500000
    total_period: total mortgage duration in months
    init_period: initial term duration in months
    init_payment: initial monthly payment amount, e.g. 1205
    init_rate: initial interest rate expressed as a decimal. e.g. 0.0146 for 1.46%
    subs_rate: subsequent interest rate expressed as a decimal. e.g. 0.0376 for 3.76%
    base_rate: Bank of England base rate. e.g. 0.001 for 0.1%
    over_payment: optional fixed monthly overpayment
    """    
    
    # create arrays for plotting
    interest_values = []
    payment_values = []
    over_payment_values = []
    principal_values = []
    interest_cum_values = []
   
    # calculate initial max overpayment
    max_monthly_overpayment = principal*0.1*(1/12)
    
    # cumulative interest
    interest_cum = 0
    
    # payments for months of initial period
    for month in range(0, init_period):
        
        # update max available monthly overpayment each year
        if month % 12 == 0:
            max_monthly_overpayment = principal*0.1*(1/12)
            
        # calculate interest
        interest = principal*(1 + init_rate)**(1/12) - principal
        
        # add to interest for this month to total interest charged
        interest_cum += interest
    
        # add interest for current month
        principal += interest
     
        # subtract payment amount from principal
        if principal < init_payment:
            principal -= principal
            #stop payments if principal paid off
            init_payment=0 
            subs_payment=0
            over_payment=0
        else:
            principal -= init_payment       
            
        # subtract overpayment from principal
        if over_payment > max_monthly_overpayment:
            over_payment = max_monthly_overpayment

            if principal < over_payment:
                principal -= principal
                init_payment=0 
                subs_payment=0
                over_payment=0
            else:
                principal -= over_payment
        else:
            if principal < over_payment:
                principal -= principal
                init_payment=0 
                subs_payment=0
                over_payment=0
            else:
                principal -= over_payment
   
        # populate arrays for plotting
        interest_values.append(interest)
        interest_cum_values.append(interest_cum)
        payment_values.append(init_payment)
        over_payment_values.append(over_payment)
        principal_values.append(principal)
   
    # calculate payments for remaining balance at new rate
    subs_payment = ((((subs_rate+base_rate)/12))*((1+((subs_rate+base_rate)/12))**(total_period-init_period)) / (((1+((subs_rate+base_rate)/12))**(total_period-init_period)) - 1))*(principal)
    
    # payments for subsequent months
    for month in range(init_period, total_period):
        
        # interest added
        interest = principal*(1 + (subs_rate+base_rate))**(1/12) - principal
        
        # add to cumulative interest 
        interest_cum += interest
            
        # add interest for current month
        principal += interest
    
        # subtract (minimum) payment amount from principal
        if principal < subs_payment:
            principal -= principal
            #stop payments if principal paid off
            init_payment=0 
            subs_payment=0
            over_payment=0
        else:
            principal -= subs_payment
            
        # subtract overpayment amount from principal
        if principal < over_payment:
            principal -= principal
            #stop payments if principal paid off
            init_payment=0 
            subs_payment=0
            over_payment=0

        else:
            principal -= over_payment
        
        # populate arrays for plotting
        interest_values.append(interest)
        interest_cum_values.append(interest_cum)
        payment_values.append(subs_payment)
        over_payment_values.append(over_payment)
        principal_values.append(principal)
        
    results = dict.fromkeys(['Interest', 'Payments', 'Overpayments', 'Principal'])
    results['Interest'] = interest_values
    results['Payments'] = payment_values
    results['Overpayments'] = over_payment_values
    results['Principal'] = principal_values
    
    return results